In [28]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime

In [29]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [30]:
# Starting URL for Weather Map API Call.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=29bca3afd9b264056d3287fa6d1ba205


In [31]:
# Create a set of 2000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [32]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [33]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

87.50020891947128 106.11603692731904
-49.82946754726746 76.07577717854412
-75.89045632451283 -7.061118192559206
77.29395695880618 21.822743610913562
-45.41752904802414 -99.40497940155036
-64.55843428819307 111.51941606525423
69.3735373492101 -53.70562037133266
51.309214437498895 -79.26435711162972
31.54647978646443 -167.85734031062543
60.50712830174564 -169.87721379336148
89.80386551649721 -22.75746999729762
-27.31023562957715 -138.88566267858968
48.20619168161835 174.52482078154577
14.910779523889062 110.8956743038965
-33.98046516341675 -74.96701736801595
-33.9118500336034 59.11928592747998
-30.02168268344265 -104.53131543717961
69.93477627101234 149.8699262915424
2.107077161682952 -38.51351801964418
81.17136148320353 -25.784861452051217
65.29988258667754 -53.13444413781441
-65.72621397935846 -115.40113209446523
-22.27425621375515 45.131929150844485
77.13887562860592 -119.94394009179341
15.576345466682398 -155.06891906340067
-74.65443054100243 27.1948970662404
77.74144972431102 12.775

-9.180156817740297 -35.445847876627795
-67.8193683630191 96.07589120563449
4.779803120977732 15.357243890783764
-55.26651182413017 -26.284875280895278
66.59070936808604 37.771168789883035
62.009703025504535 -70.47321428184432
-44.23317714020735 161.05853596445644
59.80972986838867 95.6875712041736
3.3213504736154675 -153.08441583448382
86.34368331986346 -159.56841699506865
25.254656391967018 81.86098905359688
-17.005669440883878 -146.48242008335623
5.6301925400371005 -82.18873243973124
-58.96127975927088 88.56860351974399
55.91080106199115 97.79486845346463
-18.14588694219448 30.17999386736784
69.4270626491994 -68.150897167296
44.15613501288513 -164.00707180147523
-54.072989814939234 38.111839604434294
58.81209666303769 -83.01193678935063
-38.832836472938894 47.223737883878414
28.57407308663207 89.50481386561484
-69.07559122580342 137.02262230313846
-11.96735387494573 30.586893278880922
11.746568006209316 31.25676571612783
1.6458882908207642 137.42763045795454
8.547984644303156 -80.495

In [34]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [35]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

734

In [36]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")
    
     # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather['weather'][0]['description']

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                        "Current Description": city_desc,
                         })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | khatanga
Processing Record 2 of Set 1 | mahebourg
Processing Record 3 of Set 1 | cape town
Processing Record 4 of Set 1 | longyearbyen
Processing Record 5 of Set 1 | castro
Processing Record 6 of Set 1 | albany
Processing Record 7 of Set 1 | aasiaat
Processing Record 8 of Set 1 | moose factory
Processing Record 9 of Set 1 | kapaa
Processing Record 10 of Set 1 | provideniya
Processing Record 11 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | nikolskoye
Processing Record 14 of Set 1 | qui nhon
Processing Record 15 of Set 1 | constitucion
Processing Record 16 of Set 1 | saint-philippe
Processing Record 17 of Set 1 | chokurdakh
Processing Record 18 of Set 1 | itarema
Processing Record 19 of Set 1 | maniitsoq
Processing Record 20 of Set 1 | punta arenas
Processing Record 21 of Set 1 | beroroha
Processing Record 22 of Set 1

Processing Record 40 of Set 4 | daru
Processing Record 41 of Set 4 | douentza
Processing Record 42 of Set 4 | cockburn town
Processing Record 43 of Set 4 | port elizabeth
Processing Record 44 of Set 4 | vardo
Processing Record 45 of Set 4 | nizhneyansk
City not found. Skipping...
Processing Record 46 of Set 4 | belushya guba
City not found. Skipping...
Processing Record 47 of Set 4 | treinta y tres
Processing Record 48 of Set 4 | tasiilaq
Processing Record 49 of Set 4 | mount isa
Processing Record 50 of Set 4 | rawson
Processing Record 1 of Set 5 | la paz
Processing Record 2 of Set 5 | lorengau
Processing Record 3 of Set 5 | pevek
Processing Record 4 of Set 5 | mtimbira
Processing Record 5 of Set 5 | saint-pierre
Processing Record 6 of Set 5 | husavik
Processing Record 7 of Set 5 | sechura
Processing Record 8 of Set 5 | kaitangata
Processing Record 9 of Set 5 | sfantu gheorghe
Processing Record 10 of Set 5 | kalisz
Processing Record 11 of Set 5 | sola
Processing Record 12 of Set 5 | ch

Processing Record 30 of Set 8 | vicuna
Processing Record 31 of Set 8 | san pedro
Processing Record 32 of Set 8 | zhangjiakou
Processing Record 33 of Set 8 | touros
Processing Record 34 of Set 8 | chadiza
Processing Record 35 of Set 8 | kayes
Processing Record 36 of Set 8 | kapoeta
Processing Record 37 of Set 8 | matara
Processing Record 38 of Set 8 | waitakere
Processing Record 39 of Set 8 | balikpapan
Processing Record 40 of Set 8 | acari
Processing Record 41 of Set 8 | prabumulih
Processing Record 42 of Set 8 | los llanos de aridane
Processing Record 43 of Set 8 | dolinsk
Processing Record 44 of Set 8 | akyab
Processing Record 45 of Set 8 | springdale
Processing Record 46 of Set 8 | palabuhanratu
City not found. Skipping...
Processing Record 47 of Set 8 | elmira
Processing Record 48 of Set 8 | borogontsy
Processing Record 49 of Set 8 | hualmay
Processing Record 50 of Set 8 | codrington
Processing Record 1 of Set 9 | bathsheba
Processing Record 2 of Set 9 | viedma
Processing Record 3 

Processing Record 15 of Set 12 | san jose
Processing Record 16 of Set 12 | swan hill
Processing Record 17 of Set 12 | tutoia
Processing Record 18 of Set 12 | mayo
Processing Record 19 of Set 12 | bolshiye klyuchishchi
City not found. Skipping...
Processing Record 20 of Set 12 | margate
Processing Record 21 of Set 12 | port hedland
Processing Record 22 of Set 12 | karasuk
Processing Record 23 of Set 12 | likasi
Processing Record 24 of Set 12 | belyy yar
Processing Record 25 of Set 12 | nisia floresta
Processing Record 26 of Set 12 | leningradskiy
Processing Record 27 of Set 12 | paidha
Processing Record 28 of Set 12 | andreapol
Processing Record 29 of Set 12 | quatis
Processing Record 30 of Set 12 | jardim
Processing Record 31 of Set 12 | broken hill
Processing Record 32 of Set 12 | khrebtovaya
Processing Record 33 of Set 12 | denizli
Processing Record 34 of Set 12 | celestun
Processing Record 35 of Set 12 | havre-saint-pierre
Processing Record 36 of Set 12 | sumoto
Processing Record 37

In [37]:
len(city_data)

673

In [39]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Khatanga,RU,2022-06-11 18:22:31,71.9667,102.5000,46.87,64,45,9.62,scattered clouds
1,Mahebourg,MU,2022-06-11 18:23:03,-20.4081,57.7000,73.69,73,75,10.36,broken clouds
2,Cape Town,ZA,2022-06-11 18:20:08,-33.9258,18.4232,70.12,48,0,6.91,clear sky
3,Longyearbyen,SJ,2022-06-11 18:19:57,78.2186,15.6401,39.04,70,75,9.22,broken clouds
4,Castro,BR,2022-06-11 18:23:04,-24.7911,-50.0119,53.29,59,100,5.41,overcast clouds
5,Albany,US,2022-06-11 18:23:05,42.6001,-73.9662,77.77,50,100,1.01,overcast clouds
6,Aasiaat,GL,2022-06-11 18:23:05,68.7098,-52.8699,34.20,88,82,8.37,broken clouds
7,Moose Factory,CA,2022-06-11 18:23:06,51.2585,-80.6094,57.20,77,100,10.36,overcast clouds
8,Kapaa,US,2022-06-11 18:23:06,22.0752,-159.3190,78.78,83,75,12.66,broken clouds
9,Provideniya,RU,2022-06-11 18:23:07,64.3833,-173.3000,36.84,71,53,1.92,broken clouds


In [41]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")